## 1. Motivation:  Tidy Up the Data

- In this Notebook we will explore various ways to make our data 'Tidy', following Hadley Wickham's principles.
- We assume that you are already familiar with ways too 'clean' the data.
- Now we will focus on two techniques:  `explode` and `melt`

### 1a.  Load Libraries

In [104]:
# import libraries

import pandas as pd

# supress warnings
import warnings
warnings.filterwarnings('ignore')

#### 1b. Load CSV Sheets

In [122]:
#  Beatles Spotify Data (just clean, not tidy!)
beatles_spotify_csv = 'https://raw.githubusercontent.com/RichardFreedman/Encoding_Music/refs/heads/main/02_Lab_Data/Beatles/Beatles_Spotify_2026.csv'
beatles_spotify = pd.read_csv(beatles_spotify_csv)


# Beatles Billboard Data (clean not tidy!)
beatles_billboard_clean_csv = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vTvOhZSzoa8kwkmYOzwuAGq3Piaujeabu41wVgAhSPfS0IONz2zi_nG5Km_5GO8B8P6tor0r8CacyNF/pub?output=csv'
bb_clean = pd.read_csv(beatles_billboard_clean_csv)


# and perhaps OUR Cleaned Metadata too!
our_cleaned_beatles_metadata_csv = 'https://raw.githubusercontent.com/RichardFreedman/Encoding_Music/refs/heads/main/02_Lab_Data/Beatles/our_clean_beatles_data.csv'
our_clean_beatles = pd.read_csv(our_cleaned_beatles_metadata_csv)


# and for final comparison, you can even load a completely exploded and melted version of the billboard and spotify data
beatles_bb_spotify_final = 'https://github.com/RichardFreedman/Encoding_Music/raw/refs/heads/main/02_Lab_Data/Beatles/beatles_data_combined_clean_tidy.pkl'
our_clean_beatles = pd.read_pickle(beatles_bb_spotify_final)

# 2.  Implementation

### Here is the plan . . ..

- a) Genre:  split, clean and explode
- b) Spotify:  melt the audio features
- c) Combine:  merge the dfs
- d) Think about OUR data in relation to these

In [123]:
# single row from our data looks like this:
bb_clean.iloc[100]


Title                                            Hold Me Tight
Year                                                      1963
Album.debut         UK: With the Beatles US: Meet The Beatles!
Duration                                                   152
Other.releases                                              16
Genre                                    rock & roll, pop/rock
Songwriter                              McCartney, with Lennon
Lead.vocal                                           McCartney
Top.50.Billboard                                             0
Name: 100, dtype: object

In [124]:
# confirm the data type
type(bb_clean.iloc[100]['Genre'])

str

### 2a. Tidy Genres with Explode

In [125]:
# just the genre column
bb_clean.iloc[100]['Genre']

'rock & roll, pop/rock'

In [126]:
# confirm the data type
type(bb_clean.iloc[100]['Genre'])

str

#### Split the Genre Strings!

- We can use the `str` accessor and `split(',')` to systematically divide all these long strings on the `,` and return things as a`list`:

```python
bb_clean['Genre'] = bb_clean['Genre'].str.split(', ')
```

In [127]:
bb_clean['Genre'] = bb_clean['Genre'].str.split(', ')

In [128]:
bb_clean.iloc[100]['Genre']

['rock & roll', 'pop/rock']

In [129]:
# confirm the data type
type(bb_clean.iloc[100]['Genre'])

list

In [131]:
# now the complete DF
bb_clean.head(5)

,Title,Year,Album.debut,Duration,Other.releases,Genre,Songwriter,Lead.vocal,Top.50.Billboard
0,12-Bar Original,1965,Anthology 2,174,0,[blues],"Lennon, McCartney, Harrison and Starkey",instrumental,0
1,A Day in the Life,1967,Sgt. Pepper's Lonely Hearts Club Band,335,12,"[psychedelic rock, art rock, pop/rock]",Lennon and McCartney,Lennon and McCartney,0
2,A Hard Day's Night,1964,UK: A Hard Day's Night US: 1962-1966,152,35,"[rock, electronic, pop/rock]",Lennon,"Lennon, with McCartney",8
3,A Shot of Rhythm and Blues,1963,Live at the BBC,104,0,"[r&b, pop/rock]",Thompson,Lennon,0
4,A Taste of Honey,1963,UK: Please Please Me US: The Early Beatles,163,29,"[pop/rock, jazz, stage&screen]","Scott, Marlow",McCartney,0


### Explode the lists!

- Humans can make sense of those lists.  But for various steps of grouping, filtering and making charts, it will be better to keep **one genre tag per row**.
- We can easily do this with a Pandas method called `explode().  Here is how:

```python
bb_exploded = bb_clean.explode('Genre').reset_index(drop=True)
```

In [132]:
# explode the data on the genre column, and save that as a new df
bb_exploded = bb_clean.explode('Genre').reset_index(drop=True)
bb_exploded.head()

,Title,Year,Album.debut,Duration,Other.releases,Genre,Songwriter,Lead.vocal,Top.50.Billboard
0,12-Bar Original,1965,Anthology 2,174,0,blues,"Lennon, McCartney, Harrison and Starkey",instrumental,0
1,A Day in the Life,1967,Sgt. Pepper's Lonely Hearts Club Band,335,12,psychedelic rock,Lennon and McCartney,Lennon and McCartney,0
2,A Day in the Life,1967,Sgt. Pepper's Lonely Hearts Club Band,335,12,art rock,Lennon and McCartney,Lennon and McCartney,0
3,A Day in the Life,1967,Sgt. Pepper's Lonely Hearts Club Band,335,12,pop/rock,Lennon and McCartney,Lennon and McCartney,0
4,A Hard Day's Night,1964,UK: A Hard Day's Night US: 1962-1966,152,35,rock,Lennon,"Lennon, with McCartney",8


### But now we need to Clean Again!

- Now that the genres have been split into separate rows, we need to check for all the usual problems:
- white space
- missing values (which might come from oddities in the original long strings)
- inconsistent values

In [133]:
# consider values in exploded genres for clean up:
sorted(bb_exploded['Genre'].fillna('unspecified').unique().tolist())

['acid rock',
 'acid rock[',
 'art pop',
 'art rock',
 'avant-garde',
 'avant-pop',
 'baroque pop',
 'beat',
 'blues',
 'blues rock',
 'bolero',
 'boogie-woogie',
 "children's",
 "children's music",
 'circus',
 'comedy rock',
 'country',
 'country blues',
 'country rock',
 'doo wop',
 'electronic',
 'electronic pop/rock',
 'experimental',
 'experimental music',
 'experimental pop',
 'experimental rock',
 'folk',
 'folk blues',
 'folk pop',
 'folk rock',
 'folkpop/rock',
 'gospel',
 'hard rock',
 'heavy metal',
 'heavy metal[',
 'hindustani blues',
 'indian classical',
 'indian music',
 'instrumental rock',
 'jangle pop',
 'jazz',
 'jazz fusion',
 'jazz-pop',
 'jump blues',
 'lo-fi',
 'lullaby',
 'merseybeat',
 'music hall',
 'musique concrete',
 'novelty',
 'orchestral pop',
 'outsider music',
 'pop rock',
 'pop/rock',
 'power pop',
 'progressive pop',
 'progressive rock',
 'proto-punk',
 'psychedelic folk',
 'psychedelic pop',
 'psychedelic rock',
 'r&b',
 'raga rock',
 'ragtime',
 'r

In [33]:
# and clean these.  In this case our function takes in a COMPLETE COLUMN, not individual values in cells
# note that the content between """ and """ provides the documentation of our function.  The 'doc string'

def clean_genre(genre_series):
    """
    Clean and standardize genre names.
    
    Parameters:
    genre_series: pandas Series containing genre data
    
    Returns:
    pandas Series with cleaned genre data
    """
    return (genre_series
            .str.strip('[')
            .str.strip()
            .str.replace('pop/rock', 'pop rock', regex=False)
            .str.replace('r&b', 'rhythm and blues', regex=False)
            .str.replace('rock and roll', 'rock', regex=False)
            .str.replace('rock & roll', 'rock', regex=False)
            .str.replace('jazz-pop', 'jazz pop', regex=False)
            .str.replace('experimental music', 'experimental', regex=False)
            .str.replace("children's music", "children's", regex=False)
            .str.replace("folkpop/rock", "folk pop rock", regex=False)
            .str.replace("stage&screen", "stage and screen", regex=False)
            .str.replace("electronic pop/rock", "electronic pop rock", regex=False)
            .str.replace("avant-pop", "avant pop", regex=False)
        
    )

In [81]:
# designate the series (a column)
selected_series = bb_exploded['Genre']

# pass it in to the function
bb_exploded['Genre'] = clean_genre(selected_series)

# check results
bb_exploded.head()

,Title,Year,Duration,Album.debut.UK,Album.debut.US,Other.releases,Genre,Songwriter,Lead.vocal,Top.50.Billboard
0,12-Bar Original,1965,174,Anthology 2,None,0,blues,"Lennon, McCartney, Harrison and Starkey",instrumental,0
1,A Day in the Life,1967,335,Sgt. Pepper's Lonely Hearts Club Band,None,12,psychedelic rock,Lennon and McCartney,Lennon and McCartney,0
2,A Day in the Life,1967,335,Sgt. Pepper's Lonely Hearts Club Band,None,12,art rock,Lennon and McCartney,Lennon and McCartney,0
3,A Day in the Life,1967,335,Sgt. Pepper's Lonely Hearts Club Band,None,12,pop rock,Lennon and McCartney,Lennon and McCartney,0
4,A Hard Day's Night,1964,152,A Hard Day's Night,1962-1966,35,rock,Lennon,"Lennon, with McCartney",8


In [82]:
# check distribution of genres
bb_exploded['Genre'].value_counts().head(20)

Genre
pop rock             299
rock                  96
psychedelic rock      28
folk rock             22
hard rock             20
rhythm and blues      18
blues rock            11
baroque pop           10
merseybeat            10
folk                   9
music hall             8
country                8
psychedelic pop        7
power pop              6
blues                  5
psychedelic folk       5
jazz                   5
rockabilly             5
experimental rock      4
raga rock              4
Name: count, dtype: int64

In [135]:
# save our exploded data locally as csv
bb_exploded.to_csv('beatles_billboard_clean_explode.csv')

# and here is how to reload it
bb_exploded = pd.read_csv('beatles_billboard_clean_explode.csv')

### 2b. Tidy Spotify Data with Melt

- There are LOTS of columns for the audio features! We could reshape the Audio Features with pd.melt()

- For example we might want ONE variable per column for the audio features, and then a 'value' column with the values for each feature.  



In [138]:
# inspect:
beatles_spotify.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 191 entries, 0 to 190
Data columns (total 17 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   year              191 non-null    int64  
 1   album             191 non-null    object 
 2   song              191 non-null    object 
 3   spotify url       191 non-null    object 
 4   danceability      191 non-null    float64
 5   energy            191 non-null    float64
 6   key               191 non-null    int64  
 7   loudness          191 non-null    float64
 8   mode              191 non-null    int64  
 9   speechiness       191 non-null    float64
 10  acousticness      191 non-null    float64
 11  instrumentalness  191 non-null    float64
 12  liveness          191 non-null    float64
 13  valence           191 non-null    float64
 14  tempo             191 non-null    float64
 15  duration_ms       191 non-null    int64  
 16  time_signature    191 non-null    int64  
dt

In [142]:
# the columns
beatles_spotify.columns.tolist()

['year',
 'album',
 'song',
 'spotify url',
 'danceability',
 'energy',
 'key',
 'loudness',
 'mode',
 'speechiness',
 'acousticness',
 'instrumentalness',
 'liveness',
 'valence',
 'tempo',
 'duration_ms',
 'time_signature']

In [143]:
# one row
beatles_spotify.iloc[0]

year                                                             1967
album                           Sgt. Pepper's Lonely Hearts Club Band
song                                                A Day in the Life
spotify url         https://open.spotify.com/track/0hKRSZhUGEhKU6a...
danceability                                                    0.364
energy                                                          0.457
key                                                                 4
loudness                                                      -14.162
mode                                                                0
speechiness                                                    0.0675
acousticness                                                     0.29
instrumentalness                                             0.000106
liveness                                                        0.922
valence                                                         0.175
tempo               

#### Melt in Brief

- For melting the data, we are basically taking WIDE data (lots of columns with similar data) and creating LONG data (similar variables in the same column, then the values in another.
- This is very helpful for charts, plots, and groupby operations
- we need to pick our `id_vars`, which will be the main "unmelted" identifiers, like title
- and we need to pick our `value_vars`, which are the _wide_ columns we want to assemble lengthwise
- we then pass these arguments to the `pd.melt() method, along with the original df

In [85]:
# Choose our ID Vars, or identifiers - the column to keep fixed
id_vars = ['song', 'key', 'mode', 'tempo', 'time_signature', 'duration_ms']

# Choose our value_vars, or columns to "unpivot" into a single "variable" column.
value_vars = ['danceability', 'energy', 'speechiness', 'acousticness', 'liveness', 'valence']

# Using melt
spotify_melted = pd.melt(
    beatles_spotify, # Dataset
    id_vars=id_vars, # Pivot
    value_vars=value_vars # Values
).sort_values('song').reset_index(drop=True) # Sorting

spotify_melted.rename(columns = {'variable' :  'audio_feature'}, inplace=True)

In [86]:
spotify_melted.head(20)

,song,key,mode,tempo,time_signature,duration_ms,audio_feature,value
0,A Day in the Life,4,0,163.219,4,337413,danceability,0.3640
1,A Day in the Life,4,0,163.219,4,337413,valence,0.1750
2,A Day in the Life,4,0,163.219,4,337413,energy,0.4570
3,A Day in the Life,4,0,163.219,4,337413,liveness,0.9220
4,A Day in the Life,4,0,163.219,4,337413,acousticness,0.2900
5,A Day in the Life,4,0,163.219,4,337413,speechiness,0.0675
6,A Hard Day's Night,0,1,138.514,4,154200,danceability,0.5900
7,A Hard Day's Night,0,1,138.514,4,154200,energy,0.8050
8,A Hard Day's Night,0,1,138.514,4,154200,liveness,0.0996
9,A Hard Day's Night,0,1,138.514,4,154200,valence,0.7970


## 2c. Merge the our Data

- We've done this before, but note that in this case the song column is called `song` in one df and `Title` in the other.
- `pd.merge` can handle this!

In [144]:
combined_beatles_data = pd.merge(spotify_melted, bb_exploded, left_on='song', right_on='Title', how='inner')
combined_beatles_data

,song,key,mode,tempo,time_signature,duration_ms,audio_feature,value,Unnamed: 0,Title,Year,Album.debut,Duration,Other.releases,Genre,Songwriter,Lead.vocal,Top.50.Billboard
0,A Day in the Life,4,0,163.219,4,337413,danceability,0.3640,1,A Day in the Life,1967,Sgt. Pepper's Lonely Hearts Club Band,335,12,psychedelic rock,Lennon and McCartney,Lennon and McCartney,0
1,A Day in the Life,4,0,163.219,4,337413,danceability,0.3640,2,A Day in the Life,1967,Sgt. Pepper's Lonely Hearts Club Band,335,12,art rock,Lennon and McCartney,Lennon and McCartney,0
2,A Day in the Life,4,0,163.219,4,337413,danceability,0.3640,3,A Day in the Life,1967,Sgt. Pepper's Lonely Hearts Club Band,335,12,pop/rock,Lennon and McCartney,Lennon and McCartney,0
3,A Day in the Life,4,0,163.219,4,337413,valence,0.1750,1,A Day in the Life,1967,Sgt. Pepper's Lonely Hearts Club Band,335,12,psychedelic rock,Lennon and McCartney,Lennon and McCartney,0
4,A Day in the Life,4,0,163.219,4,337413,valence,0.1750,2,A Day in the Life,1967,Sgt. Pepper's Lonely Hearts Club Band,335,12,art rock,Lennon and McCartney,Lennon and McCartney,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2437,Your Mother Should Know,0,1,117.613,4,148413,speechiness,0.0332,661,Your Mother Should Know,1967,Magical Mystery Tour,149,13,pop/rock,McCartney,McCartney,0
2438,Your Mother Should Know,0,1,117.613,4,148413,danceability,0.6980,658,Your Mother Should Know,1967,Magical Mystery Tour,149,13,music hall,McCartney,McCartney,0
2439,Your Mother Should Know,0,1,117.613,4,148413,danceability,0.6980,659,Your Mother Should Know,1967,Magical Mystery Tour,149,13,vaudeville rock,McCartney,McCartney,0
2440,Your Mother Should Know,0,1,117.613,4,148413,danceability,0.6980,660,Your Mother Should Know,1967,Magical Mystery Tour,149,13,psychedelic pop,McCartney,McCartney,0


Final Clean Up of Columns

In [145]:
# force cols to lower case for simplicity
combined_beatles_data.rename(columns=str.lower, inplace=True)
# drop redundant columns
cols_to_drop = ['title', 'duration', 'other.releases']

# this is called 'list comprehension' and will create a list of those NOT in the exclusion list!
cols_to_keep = [col for col in combined_beatles_data.columns if col not in cols_to_drop]

# now a new df with only the colums we want
combined_beatles_data_brief = combined_beatles_data[cols_to_keep]


In [101]:
combined_beatles_data_brief.head(5)

,song,key,mode,tempo,time_signature,duration_ms,audio_feature,value,year,album.debut.uk,album.debut.us,genre,songwriter,lead.vocal,top.50.billboard
0,A Day in the Life,4,0,163.219,4,337413,danceability,0.364,1967,Sgt. Pepper's Lonely Hearts Club Band,None,psychedelic rock,Lennon and McCartney,Lennon and McCartney,0
1,A Day in the Life,4,0,163.219,4,337413,danceability,0.364,1967,Sgt. Pepper's Lonely Hearts Club Band,None,art rock,Lennon and McCartney,Lennon and McCartney,0
2,A Day in the Life,4,0,163.219,4,337413,danceability,0.364,1967,Sgt. Pepper's Lonely Hearts Club Band,None,pop rock,Lennon and McCartney,Lennon and McCartney,0
3,A Day in the Life,4,0,163.219,4,337413,valence,0.175,1967,Sgt. Pepper's Lonely Hearts Club Band,None,psychedelic rock,Lennon and McCartney,Lennon and McCartney,0
4,A Day in the Life,4,0,163.219,4,337413,valence,0.175,1967,Sgt. Pepper's Lonely Hearts Club Band,None,art rock,Lennon and McCartney,Lennon and McCartney,0


In [103]:
# save the result as 'pickled' file, to preserve all data types!

combined_beatles_data_brief.to_pickle('beatles_combined_clean_exploded_brief.pkl')

## 2d Now Tidy OUR Clean Metadata, then decide on how to merge with Billboard or Spotify!

- `Split` the genres and contexts
- `Explode` these?
- `Merge` on titles, and decide 'how'
- Then we can think about correlations between . . . . Genres and Audio, Popularity and Contexts, Popularity and Genres, People and Albums, etc.


3.  Interpretation:

What did you learn